In [221]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
import pandas as pd

In [222]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_s'

In [223]:
headers = {
    'User-Agent': USER_AGENT,
}

**Парсинг head hanter**

In [224]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      #'vacancy_salary': vacancy_salary,
      'salary_min': min_salary,
      'salary_max': max_salary,
      'salary_currency': currency_salary,
      'vacancy_link': vacancy_link,
      'site': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [225]:
#result = parse_hh(URL_FIRST_PAGE_HH, headers)

In [226]:
#result

**Парсинг superjob**

In [227]:
def _parser_superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy, \
        'profession_only': '1', \
        'geo[c][0]': '15', \
        'geo[c][1]': '1', \
        'geo[c][2]': '9', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    
    link = 'https://www.superjob.ru/vacancy/search/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
    
        page_block = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})
                        
            for item in vacancy_items:
                vacancy_date.append(_parser_item_superjob(item))
                
    return vacancy_date

In [228]:
def _parser_item_superjob(item):

    vacancy_date = {}
    
    # vacancy_name
    vacancy_name = item.find('span', {'class': '_26gg2 _3oXMw _2LaRg hbKbL rIDaO oDIMq _33qju _1ZV-S'}) \
                            .findChildren()[1] \
                            .getText() \
                            .split(',')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-1].getText()
    #else:
     #   vacancy_name = vacancy_name[0].getText()
    vacancy_date['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'})
    
    if not company_name:
        company_name = item.findParent() \
                            .find('span', {'class': 'f-test-text-vacancy-item-company-name'}) \
                            .getText()
    else:
        company_name = company_name.getText()
    
    vacancy_date['company_name'] = company_name
    
    # city
    company_location = item.find('span', {'class': 'f-test-text-company-item-location'}) \
                            .findChildren()[1] \
                            .getText() \
                            .split(',')
    
    vacancy_date['city'] = company_location[0]
    
    #metro station
    if len(company_location) > 1:
        metro_station = company_location[1]
    else:
        metro_station = None
    
    vacancy_date['metro_station'] = metro_station
    
    #salary
    salary = item.find('div', {'class': 'f-test-text-company-item-salary'}) \
                  .findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_currency = salary[-1].getText()
        is_check_sarary = item.find('div', {'class': 'f-test-text-company-item-salary'}) \
                                .getText() \
                                .replace(u'\xa0', u' ') \
                                .split(' ', 1)[0]
        if is_check_sarary == 'до' or len(salary) == 2:
            salary_min = None
            salary_max = int(salary[0].getText() \
                                        .replace(u'\xa0', u''))
        elif is_check_sarary == 'от':
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = None
        
        elif is_check_sarary == 'По':
            
            salary_min = None
            
            salary_max = None

        else:
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = int(salary[2].getText() \
                                         .replace(u'\xa0', u''))           
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    
    # link
    vacancy_link = item.find_all('a')
    
    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']
    
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru{vacancy_link }'
    
    # site
    vacancy_date['site'] = 'www.superjob.ru'
    
    return vacancy_date

In [229]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(parse_hh(URL_FIRST_PAGE_HH, headers))
    vacancy_date.extend(_parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)

    return df

In [230]:
vacancy = 'Python'
df = parser_vacancy(vacancy)
df

Cтраница №1, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_s
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_s=&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_s=&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_s=&page=3&hhtmFrom=vacancy_search_list
Парсинг завершен


,vacancy_name,salary_min,salary_max,salary_currency,vacancy_link,site,company_name,city,metro_station
0,Разработчик Python,120000.0,NaN,руб.,https://spb.hh.ru/vacancy/76076341?from=vacanc...,hh.ru,NaN,NaN,NaN
1,Team Lead Python,400000.0,NaN,руб.,https://spb.hh.ru/vacancy/76930290?from=vacanc...,hh.ru,NaN,NaN,NaN
2,Middle Python developer (Казань),160000.0,NaN,руб.,https://spb.hh.ru/vacancy/76115939?from=vacanc...,hh.ru,NaN,NaN,NaN
3,Junior/Middle Backend-разработчик на Ruby,110000.0,150000.0,руб.,https://spb.hh.ru/vacancy/76964796?from=vacanc...,hh.ru,NaN,NaN,NaN
4,Python разработчик,NaN,NaN,None,https://spb.hh.ru/vacancy/75951525?from=vacanc...,hh.ru,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
79,"C++ Developer (Middle, Senior)",NaN,NaN,None,https://spb.hh.ru/vacancy/76124182?from=vacanc...,hh.ru,NaN,NaN,NaN
80,[Python],NaN,NaN,По договорённости,https://www.superjob.ru/clients/bank-vtb-23557...,www.superjob.ru,Банк ВТБ,Москва,NaN
81,[Python],NaN,NaN,По договорённости,https://www.superjob.ru/clients/megapolis-1211...,www.superjob.ru,МЕГАПОЛИС,Москва,NaN
82,[Python],NaN,NaN,По договорённости,https://www.superjob.ru/clients/bank-vtb-23557...,www.superjob.ru,Банк ВТБ,Москва,NaN


In [231]:
df.to_csv (r'data.csv', sep=',', index= False )